In [2]:
import py_entitymatching as em
import pandas as pd
import os
import sys
from timeit import default_timer as timer

source1 = 'source1_cleaned.csv'
source2 = 'source2_cleaned.csv'

# Read the data
A = em.read_csv_metadata(source1)
B = em.read_csv_metadata(source2)

# Set the metadata
em.set_key(A, 'ID')
em.set_key(B, 'ID')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


True

In [3]:
# Load the golden data
S = em.read_csv_metadata('TuplesForLabeling_cleaned.csv', key='_id', ltable=A, rtable=B, 
                         fk_ltable='ltable_ID', fk_rtable='rtable_ID')

# Split S into development set (I) and evaluation set (J)
IJ = em.split_train_test(S, train_proportion=0.7, random_state=42)
I = IJ['train']
J = IJ['test']

# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0, criterion='gini', class_weight='balanced')
svm = em.SVMMatcher(name='SVM', kernel='linear', random_state=0)
rf = em.RFMatcher(name='RF', n_estimators=50, criterion='gini', class_weight='balanced', random_state=0)
lg = em.LogRegMatcher(name='LogReg', penalty='l2', class_weight='balanced', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

Metadata file is not present in the given path; proceeding to read the csv file.


In [7]:
# Generate features
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

# Drop unnecesary features. Here, we performed multiple iterations and tried out different combinations
feature_table = feature_table.drop([0,1,2,3,17,18,19,20,21,22,23,24,25,30,34,35,36,37,38])

# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='match',
                            show_progress=False)

H = em.impute_table(H, 
                exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'match'],
                strategy='mean')

In [9]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0, criterion='gini', class_weight='balanced')
svm = em.SVMMatcher(name='SVM', kernel='linear', random_state=0)
rf = em.RFMatcher(name='RF', n_estimators=50, criterion='gini', class_weight='balanced', random_state=0)
lg = em.LogRegMatcher(name='LogReg', penalty='l2', class_weight='balanced', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'match'],
        k=5,
        target_attr='match', metric_to_select_matcher='f1', random_state=42)

print(result['cv_stats'])
print(result['drill_down_cv_stats']['f1'])

/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)
/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)
/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)
/usr/local/lib/pyth

        Matcher  Average precision  Average recall  Average f1
0  DecisionTree           0.592857        0.590476    0.564267
1            RF           0.966667        0.612381    0.707459
2           SVM           0.852381        0.590476    0.637121
3        LinReg           0.893333        0.566667    0.675556
4        LogReg           0.595311        0.886667    0.690131
5    NaiveBayes           0.609286        0.824762    0.691784
           Name  \
0  DecisionTree   
1            RF   
2           SVM   
3        LinReg   
4        LogReg   
5    NaiveBayes   

                                                                         Matcher  \
0          <py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x10f4e9cf8>   
1          <py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x10f4e9978>   
2        <py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x10f4e9898>   
3  <py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x10f4e9e48>   
4  <p

In [11]:
# Debugging Random forest
# Fit the random forest to the feature vectors
rf.fit(table=H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'match'], target_attr='match')

# Use the SVM matcher to predict if tuple pairs match
predictions = rf.predict(table=H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'match'], target_attr='predicted_labels', 
           append=True, inplace=False)

eval_result = em.eval_matches(predictions, 'match', 'predicted_labels')
em.print_eval_summary(eval_result)

Precision : 100.0% (35/35)
Recall : 100.0% (35/35)
F1 : 100.0%
False positives : 0 (out of 35 positive predictions)
False negatives : 0 (out of 315 negative predictions)


In [12]:
# Check the bad predictions (if any), split train set and debug using GUI
print (predictions[predictions['match'] != predictions['predicted_labels']])




Empty DataFrame
Columns: [_id, ltable_ID, rtable_ID, Name_Name_jac_qgm_3_qgm_3, Name_Name_cos_dlm_dc0_dlm_dc0, Name_Name_mel, Name_Name_lev_dist, Name_Name_lev_sim, Author_Author_jac_qgm_3_qgm_3, Author_Author_cos_dlm_dc0_dlm_dc0, Author_Author_jac_dlm_dc0_dlm_dc0, Author_Author_mel, Author_Author_lev_dist, Author_Author_lev_sim, Author_Author_nmw, Author_Author_sw, Publishing_Date_Publishing_Date_lev_sim, Publishing_Date_Publishing_Date_jar, Publishing_Date_Publishing_Date_jwn, Publishing_Date_Publishing_Date_exm, Pages_Pages_exm, Pages_Pages_anm, Pages_Pages_lev_dist, match, predicted_labels]
Index: []

[0 rows x 25 columns]


In [13]:
# Try out trigger rules 
# Triggers
# Use the constructor to create a trigger
mt = em.MatchTrigger()
mt.add_cond_rule(['Name_Name_cos_dlm_dc0_dlm_dc0(ltuple, rtuple) >= 0.9', 'Author_Author_cos_dlm_dc0_dlm_dc0(ltuple, rtuple) >= 0.9'], feature_table)
mt.add_cond_status(True)
mt.add_action(1)

preds = mt.execute(input_table=predictions, label_column='predicted_labels', inplace=True)
predictions.head()
eval_result = em.eval_matches(predictions, 'match', 'predicted_labels')
em.print_eval_summary(eval_result)

Precision : 100.0% (35/35)
Recall : 100.0% (35/35)
F1 : 100.0%
False positives : 0 (out of 35 positive predictions)
False negatives : 0 (out of 315 negative predictions)


In [16]:
# Split H into P and Q
PQ = em.split_train_test(H, train_proportion=0.5, random_state=0)
P = PQ['train']
Q = PQ['test']

# Debug RF matcher using GUI
em.vis_debug_rf(rf, P, Q, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'match'],
        target_attr='match')

# Repeat the above steps multiple times to debug and get the best out of the matcher